<a href="https://colab.research.google.com/github/rolitta/Deeplearning_classification/blob/main/Satellite_imagery_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install patchify

In [10]:
import os
import cv2
from PIL import Image
import numpy as np
from patchify import patchify
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from matplotlib import pyplot as plt
import random

In [11]:
minmaxscaler = MinMaxScaler()

In [4]:
!ls -lah '/content/drive/MyDrive/Colab Notebooks/satellite/Semantic segmentation dataset'

total 40K
-rw------- 1 root root  548 May 29  2020  classes.json
-rw------- 1 root root 6.1K May 31  2023  .DS_Store
drwx------ 2 root root 4.0K May 31  2023 'Tile 1'
drwx------ 2 root root 4.0K May 31  2023 'Tile 2'
drwx------ 2 root root 4.0K May 31  2023 'Tile 3'
drwx------ 2 root root 4.0K May 31  2023 'Tile 4'
drwx------ 2 root root 4.0K May 31  2023 'Tile 5'
drwx------ 2 root root 4.0K May 31  2023 'Tile 6'
drwx------ 2 root root 4.0K May 31  2023 'Tile 7'
drwx------ 2 root root 4.0K May 31  2023 'Tile 8'


In [5]:
dataset_root_folder = '/content/drive/MyDrive/Colab Notebooks/satellite/'

In [6]:
dataset_name = "Semantic segmentation dataset"

In [7]:
from inspect import isgeneratorfunction
for path, subdirs, files in os.walk(os.path.join(dataset_root_folder, dataset_name)):
  dir_name = path.split(os.path.sep)[-1]
  #print (dir_name)
  if dir_name == 'images':
    images = os.listdir(path)
    for i,img_name in enumerate(images):
      if (img_name.endswith ('.jpg')):
        # print(img_name)
        a = True


In [15]:
image_dataset = []
mask_dataset = []

image_patch_size = 256

for image_type in ['images', 'masks']:
  if image_type == 'images':
    image_extension = 'jpg'
  elif image_type == 'masks':
    image_extension = 'png'

  for tile_id in range(1,8):
    for image_id in range(1,20):
      image = cv2.imread(f'{dataset_root_folder}/{dataset_name}/Tile {tile_id}/{image_type}/image_part_00{image_id}.{image_extension}',1)
      if image is not None:
        if image_type == 'masks':
          image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # print(image.shape)
        size_x = (image.shape[1]//image_patch_size)*image_patch_size
        size_y = (image.shape[0]//image_patch_size)*image_patch_size
        image = Image.fromarray(image)
        image = image.crop((0,0, size_x, size_y))
        # print("({}, {})".format(image.size[0],image.size[1]))
        image = np.array(image)
        patched_images = patchify(image, (image_patch_size,  image_patch_size, 3), step = image_patch_size)
        #print(len(patched_images))
        #print(patched_images.shape)
        for i in range(patched_images.shape[0]):
            for j in range(patched_images.shape[1]):
              if image_type == 'images':
                individual_patched_image = patched_images[i,j,:,:]
                #print(individual_patched_image.shape)
                individual_patched_image = minmaxscaler.fit_transform(individual_patched_image.reshape(-1, individual_patched_image.shape[-1])).reshape(individual_patched_image.shape)
                individual_patched_image = individual_patched_image[0]
                #print(individual_patched_image.shape)
                image_dataset.append(individual_patched_image)
              elif image_type == 'masks':
                individual_patched_mask = patched_images[i,j,:,:]
                individual_patched_mask = individual_patched_mask[0]
                mask_dataset.append(individual_patched_mask)
